In [1]:
from tkinter import *
from tkinter import ttk
import numpy as np
import pandas as pd
from scipy.stats import norm
from yahoofinancials import YahooFinancials
import plotly.express as px
import scipy.optimize as opt
import pandas_datareader.data as web
import datetime as date
import mplfinance as mpf
import yfinance as yf
from scipy.stats import norm
import matplotlib.pyplot as plt
import data as dt
import Funciones as f
from scipy.stats import norm
import scipy.stats as sta

Basic Materials


100%|██████████| 13/13 [00:09<00:00,  1.36it/s]


In [2]:
tickerr = dt.Tickets[:10]

In [3]:
#tickerr = ["AAPL", "FB", "SHOP", "C", "AMZN", "MSFT", "EPD", "KO", "TSLA","WMT"]
prices = web.DataReader(name = tickerr, data_source = "yahoo",
              start= "2022-01-01")["Adj Close"].dropna()

In [4]:
#pip install --upgrade yfinance

In [5]:
resultado=np.zeros(len(tickerr))
for i in range(len(tickerr)):
    datos=yf.download(tickerr[i], period='1d', interval='1m')
    price=datos.iloc[-1,3]
    resultado[i]=price

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [6]:
test = prices
mausherramienta = test.iloc[-24:,:]

In [7]:
prices.iloc[-25,:] = resultado
resultado

array([ 67.98999786, 309.72000122,  72.66999817,  16.70000076,
       103.80999756,  41.90999985, 244.91999817,  74.51999664,
        66.13999939, 242.91999817])

In [8]:
#Analisis RSI 
# data = yf.download(tickers, period='1d', interval='1m')
data = prices.iloc[:-25,:]

RSII = pd.DataFrame()
for i in range(len(tickerr)):
    
    delta = data.iloc[:,i].diff(1)
    delta.dropna(inplace=True)

    positive = delta.copy()
    negative = delta.copy()

    positive[positive < 0 ]=0
    negative[negative > 0 ]=0
    days = 14
    avrg_gain = positive.rolling(window=days).mean()
    avrg_loss = abs(negative.rolling(window=days).mean())

    rltv_strength = avrg_gain/avrg_loss
    RSI = 100-(100/(1+rltv_strength))

    combined = pd.DataFrame()
    combined["Close"] =  data.iloc[:,i]
    combined["RSI"] = RSI
    RSII[i] = RSI

    
RSII  = pd.DataFrame(RSII)
RSII.columns = tickerr
RSI_HOY = RSII.iloc[len(RSII)-1,:]

In [9]:
RSI_HOY=pd.DataFrame(RSI_HOY)

In [10]:
ws  = Tk()
ws.title('RSI Analysis')
ws.geometry('500x500')
#ws['bg'] = '#AC99F2'
game_frame = Frame(ws)
game_frame.pack()
my_game = ttk.Treeview(game_frame)
my_game['columns'] = ('TICKER','STATUS','PRICE')

my_game.column("#0", width=0,  stretch=NO)
my_game.column("TICKER",anchor=CENTER, width=80)
my_game.column("STATUS",anchor=CENTER,width=80)
my_game.column("PRICE",anchor=CENTER,width=80)

my_game.heading("#0",text="",anchor=CENTER)
my_game.heading("TICKER",text="TICKER",anchor=CENTER)
my_game.heading("STATUS",text="STATUS",anchor=CENTER)
my_game.heading("PRICE",text="PRICE",anchor=CENTER)

for i in range(len(tickerr)-1):
    if(RSI_HOY.iloc[i,0]<30):
        my_game.insert(parent='',index='end',text='', values=(tickerr[i],'BUY',np.round((resultado[i]),4)))
    if(RSI_HOY.iloc[i,0]>70):
        my_game.insert(parent='',index='end',text='', values=(tickerr[i],'SELL',np.round((resultado[i]),4)))
    if((RSI_HOY.iloc[i,0]>=30) & (RSI_HOY.iloc[i,0]<=70)):
        my_game.insert(parent='',index='end',text='', values=(tickerr[i],'HOLD',np.round((resultado[i]),4)))

my_game.pack()

import tkinter

Etiqueta = ttk.Label(ws,text='Select your stock')
Etiqueta.pack()

n = tkinter.StringVar(ws)
n.set(tickerr[0])

seleccion = tkinter.OptionMenu(ws,n,*tickerr)
seleccion.pack()

def Close():
    a = n.get()
    ws.destroy()

exit_button = tkinter.Button(ws, text="Choose", command= lambda :Close())
exit_button.pack(pady=20)

    
ws.mainloop()


activo = n.get()
if (RSI_HOY.T[activo][0]<30):
    Eleccion = 'Call'
elif(RSI_HOY.T[activo][0]>70):
    Eleccion = 'Put'
else:
    Eleccion = 'Call'


In [11]:
np.zeros(len(activo))

array([0., 0., 0., 0.])

In [12]:
resultado=np.zeros(len(activo))
datos=yf.download(activo, period='1d', interval='1m')
price=datos.iloc[-1,3]
st = price   

[*********************100%***********************]  1 of 1 completed


In [13]:
def griegas(r,sigma,st,k,T=1/12,t=0):
    d_c = f.delta_call(St,k,T,t,r,sigma)
    d_p = f.delta_put(St,k,T,t,r,sigma)
    v   = f.vega(St,k,T,t,r,sigma)
    t_c = f.theta_call(St,k,T,t,r,sigma)
    t_p = f.theta_put(St,k,T,t,r,sigma)
    r_c = f.rho_call(St,k,T,t,r,sigma)
    r_p = f.rho_put(St,k,T,t,r,sigma)
    g   = f.gamma(St,k,T,t,r,sigma)
    van = f.vanna(St,k,T,t,r,sigma)
    vol = f.volga(St,k,T,t,r,sigma)
    col = f.color(St,k,T,t,r,sigma)
    
    
    return d_c,d_p,v,t_c,t_p,r_c,r_p,g,van,vol,col

In [14]:
#ITERACIÓN
import time 


contador = 0
while contador < 5:
    

    resultado=np.zeros(len(activo))
    datos=yf.download(activo, period='1d', interval='1m')
    price=datos.iloc[-1,3]
    st = price 

    r = .0852
    T = 1
    t = 0
    ret_log = np.log(datos.iloc[:,3]/datos.iloc[:,3].shift(1)).dropna()
    cambio_porcentual_anual = ret_log.iloc[-1]*(252)
    sigma = np.std(ret_log)*np.sqrt(252)
    St = st 
    k = [St,St*(1+sigma),St*(1+2*sigma),St*(1-sigma),St*(1-2*sigma)]

    d1= [(np.log(St/k[i])+(r +0.5*sigma**2)*(T-t))/(sigma*np.sqrt(T-t)) for i in range(0,len(k))]
    d2= [(np.log(St/k[i])+(r -0.5*sigma**2)*(T-t))/(sigma*np.sqrt(T-t)) for i in range(0,len(k))]

    precio_justo_call = [st*sta.norm.cdf(d1[i],0,1)-k[i]*np.exp(-r*(T-t))*sta.norm.cdf(d2[i],0,1) for i in range(0,len(d1))]
    precio_justo_put = [k[i]*np.exp(-r*(T-t))*sta.norm.cdf(-d2[i],0,1)-st*sta.norm.cdf(-d1[i],0,1) for i in range(0,len(d1))]


    k = np.round(k,3)
    precio_justo_call = precio_justo_call
    precio_justo_put = precio_justo_put


    a = [griegas(r/12,sigma,st,k[i],T=1/12,t=0) for i in range(0,len(k))]
    a = a


    k1 = [0,k[0],precio_justo_call[0],precio_justo_put[0],a[0][0],a[0][1],a[0][2],a[0][3],a[0][4],
         a[0][5],a[0][6],a[0][7],a[0][8],a[0][9],a[0][10]]

    k2 = [0,k[1],precio_justo_call[1],precio_justo_put[1],a[1][0],a[1][1],a[1][2],a[1][3],a[1][4],
         a[1][5],a[1][6],a[1][7],a[1][8],a[1][9],a[1][10]]

    k3 = [0,k[2],precio_justo_call[2],precio_justo_put[2],a[2][0],a[2][1],a[2][2],a[2][3],a[2][4],
         a[2][5],a[2][6],a[2][7],a[2][8],a[2][9],a[2][10]]

    k4 = [0,k[3],precio_justo_call[3],precio_justo_put[3],a[3][0],a[3][1],a[3][2],a[3][3],a[3][4],
         a[3][5],a[3][6],a[3][7],a[3][8],a[3][9],a[3][10]]


    k5 = [0,k[4],precio_justo_call[4],precio_justo_put[4],a[4][0],a[4][1],a[4][2],a[4][3],a[4][4],
         a[4][5],a[4][6],a[4][7],a[4][8],a[4][9],a[4][10]]


    ws  = Tk()
    ws.title('Griegas '+activo)

    #ws['bg'] = '#AC99F2'
    game_frame = Frame(ws)
    game_frame.pack()
    my_game = ttk.Treeview(game_frame)
    my_game['columns'] = ('p','k','call','put','Delta_Call','Delta_put','Vega','Theta_call'
                               ,'Theta_put','rho_call','rho_put','gamma',
                              'vanna','volga','color')

    my_game.column("p", width=0,  stretch=NO)
    my_game.column("k",anchor=CENTER, width=80)
    my_game.column("call",anchor=CENTER, width=80)
    my_game.column("put",anchor=CENTER, width=80)
    my_game.column("Delta_Call",anchor=CENTER, width=80)
    my_game.column("Delta_put",anchor=CENTER,width=80)
    my_game.column("Vega",anchor=CENTER,width=80)
    my_game.column("Theta_call",anchor=CENTER,width=80)
    my_game.column("Theta_put",anchor=CENTER,width=80)
    my_game.column("rho_call",anchor=CENTER,width=80)
    my_game.column("rho_put",anchor=CENTER,width=80)
    my_game.column("gamma",anchor=CENTER,width=80)
    my_game.column("vanna",anchor=CENTER,width=80)
    my_game.column("volga",anchor=CENTER,width=80)
    my_game.column("color",anchor=CENTER,width=80)

    my_game.heading("p",text=" ",anchor=CENTER)
    my_game.heading("k",text="k",anchor=CENTER)
    my_game.heading("call",text="call",anchor=CENTER)
    my_game.heading("put",text="put",anchor=CENTER)
    my_game.heading("Delta_Call",text="Delta_Call",anchor=CENTER)
    my_game.heading("Delta_put",text="Delta_put",anchor=CENTER)
    my_game.heading("Vega",text="Vega",anchor=CENTER)
    my_game.heading("Theta_call",text="Theta_call",anchor=CENTER)
    my_game.heading("Theta_put",text="Theta_put",anchor=CENTER)
    my_game.heading("rho_call",text="rho_call",anchor=CENTER)
    my_game.heading("rho_put",text="rho_put",anchor=CENTER)
    my_game.heading("gamma",text="gamma",anchor=CENTER)
    my_game.heading("vanna",text="vanna",anchor=CENTER)
    my_game.heading("volga",text="volga",anchor=CENTER)
    my_game.heading("color",text="color",anchor=CENTER)

    my_game.insert(parent='',index='end',text='', values=(k1))
    my_game.insert(parent='',index='end',text='', values=(k2))
    my_game.insert(parent='',index='end',text='', values=(k3))
    my_game.insert(parent='',index='end',text='', values=(k4))
    my_game.insert(parent='',index='end',text='', values=(k5))
    my_game.pack()
    
    # Se actualizara la tabla 
    def cerrar():
        ws.destroy()
    
    ws.after(90000,cerrar)
    contador = contador+1 
    ws.mainloop()

    
        
    #ws.destroy()
    
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Backtest

In [15]:
k,precio_justo_call,precio_justo_put


(array([16.7  , 16.955, 17.21 , 16.445, 16.19 ]),
 [1.3639124375549763,
  1.1299664661515187,
  0.896029393204218,
  1.5978585160166237,
  1.8318045949584878],
 [4.817302799560529e-10,
  1.0802089896251897e-07,
  9.114016216543036e-06,
  7.595561408333749e-13,
  3.9869345696562005e-16])

In [16]:
Delta_1 = [a[i][0:2] for i in range (0,5)]

In [17]:
Gamma_1 = [a[i][7] for i in range (0,5)]

In [18]:
Delta_1[0][0]

0.554315098882501

In [19]:
Delta_1 = [a[i][0:2] for i in range (0,5)]
Gamma_1 = [a[i][7] for i in range (0,5)]
Test = pd.DataFrame([st,k[0],precio_justo_call[0],precio_justo_put[0],Delta_1[0][0],Delta_1[0][1],Gamma_1[0]]).T
#Test.rename(columns = {0:'St', 1:'K',2:'Call',3:'Put',4:'Delta_call',5:'Delta_put',6:'Gamma'}, inplace = True)
Test

,0,1,2,3,4,5,6
0,16.700001,16.7,1.363912,4.817303e-10,0.554315,-0.445685,0.395239


In [20]:
lista = pd.DataFrame([st,k[0],precio_justo_call[0],precio_justo_put[0],Delta_1[0][0],Delta_1[0][1],Gamma_1[0]]).T


In [21]:
np.zeros(len(mausherramienta[activo])+1)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [22]:
def back_test(k, precio_justo_call, precio_justo_put):
    r = .0852
    T = 24/252
    t = 0
    ret_log = np.log(mausherramienta[activo]/mausherramienta[activo].shift(1)).dropna()
    cambio_porcentual_anual = ret_log.iloc[-1]*(252)
    sigma = np.std(ret_log)*np.sqrt(252)
    CALIZ = np.zeros(7)
    CALIZ = pd.DataFrame(CALIZ)
    
    for i in range (1,len(mausherramienta[activo])+1):
        st = mausherramienta[activo][-i]
        k_ = k
        call = precio_justo_call
        put = precio_justo_put
        Delta_call = f.delta_call(st,k_,T,t,r,sigma)
        Delta_put =  f.delta_put(st,k_,T,t,r,sigma)
        gamma = f.gamma(st,k_,T,t,r,sigma)
        lista = pd.DataFrame([st,k_,call,put,Delta_call,Delta_put,gamma])
        CALIZ[i] = lista
        T=T-(1/252)


    CALIZ.drop(0, axis=1, inplace=True)
    CALIZ = CALIZ.T
    CALIZ.rename(columns = {0:'St', 1:'K',2:'Call',3:'Put',4:'Delta_call',5:'Delta_put',6:'Gamma'}, inplace = True)
    
    return CALIZ

In [23]:
K_1 = back_test(k[0], precio_justo_call[0], precio_justo_put[0])
K_2 = back_test(k[1], precio_justo_call[1], precio_justo_put[1])
K_3 = back_test(k[2], precio_justo_call[2], precio_justo_put[2])
K_4 = back_test(k[3], precio_justo_call[3], precio_justo_put[3])
K_5 = back_test(k[4], precio_justo_call[4], precio_justo_put[4])

In [24]:
K_1

,St,K,Call,Put,Delta_call,Delta_put,Gamma
1,16.700001,16.7,1.363912,4.817303e-10,0.551642,-4.483582e-01,3.955952e-01
2,17.650000,16.7,1.363912,4.817303e-10,0.747296,-2.527038e-01,3.195888e-01
3,18.450001,16.7,1.363912,4.817303e-10,0.867965,-1.320347e-01,2.138272e-01
4,18.750000,16.7,1.363912,4.817303e-10,0.903499,-9.650129e-02,1.709794e-01
5,19.330000,16.7,1.363912,4.817303e-10,0.950138,-4.986162e-02,1.029079e-01
6,19.480000,16.7,1.363912,4.817303e-10,0.961279,-3.872082e-02,8.392475e-02
7,19.910000,16.7,1.363912,4.817303e-10,0.979734,-2.026647e-02,4.896468e-02
8,19.930000,16.7,1.363912,4.817303e-10,0.982683,-1.731716e-02,4.283092e-02
9,20.010000,16.7,1.363912,4.817303e-10,0.986715,-1.328466e-02,3.410779e-02
10,20.100000,16.7,1.363912,4.817303e-10,0.990301,-9.698580e-03,2.594922e-02


Coberturas 
opcciones call o put 

Segunda tabla CALL

In [25]:
def segunda_tabla_call(K_1):
    
    dif_delta1CALL = [K_1["Delta_call"][i+1]-K_1["Delta_call"][i] for i in range(1,len(K_1))]
    
    factor = [1.2 if K_1["Gamma"][i]>=0.8 else 0.8 if K_1["Gamma"][i]<=0.4 else 1 for i in range(1, len(K_1))]
    
    
    compra_activo=[K_1["St"].iloc[0]*K_1["Delta_call"].iloc[0]]
    
    for i in range(len(dif_delta1CALL)):
        compra_activo.append(K_1["St"][i+1]*dif_delta1CALL[i])
        
    
    Activo_acum = [K_1["Delta_call"].iloc[0]*factor[0]]

    for i in range(len(dif_delta1CALL)):
        Activo_acum.append(Activo_acum[i]+dif_delta1CALL[i]*factor[i])    
    
    
    costo_acum = [compra_activo[0]]

    for i in range(len(dif_delta1CALL)):
        costo_acum.append(costo_acum[i]+compra_activo[i+1])
        
    ganancia_diaria= [np.max((K_1["St"].iloc[i]-K_1["K"].iloc[i]),0) for i in range(len(K_1))]
    
    costo_total=costo_acum[-1]
    
    primas=K_1["Call"]
    primas=-primas.iloc[-1]
    
    total=costo_total+primas
    
    no_cubrir=K_1["St"].iloc[-1]
    
    beneficio=no_cubrir-total
    
    f=np.zeros(len(dif_delta1CALL)+1)
    
    for i in range(len(dif_delta1CALL)):
        f[i+1]=dif_delta1CALL[i]
    
    lista = pd.DataFrame([f,compra_activo,Activo_acum,costo_acum,ganancia_diaria])
    
    lista = lista.T
    lista.rename(columns = {0:'Diff', 1:'compra_activo',2:'Activo_acum',3:'Acum_dinero',4:'Ganancia_diaria'}, inplace = True)
    
    
    return lista, costo_total,primas,total,no_cubrir,beneficio

In [26]:
st_k1_call=segunda_tabla_call(K_1)
st_k2_call=segunda_tabla_call(K_2)
st_k3_call=segunda_tabla_call(K_3)
st_k4_call=segunda_tabla_call(K_4)
st_k5_call=segunda_tabla_call(K_5)

In [27]:
st_k1_call[0]

,Diff,compra_activo,Activo_acum,Acum_dinero,Ganancia_diaria
0,0.000000e+00,9.212418,0.441313,9.212418,7.629395e-07
1,1.956544e-01,3.267429,0.597837,12.479846,9.499996e-01
2,1.206692e-01,2.129811,0.694372,14.609657,1.750001e+00
3,3.553340e-02,0.655591,0.722799,15.265248,2.050000e+00
4,4.663967e-02,0.874494,0.760111,16.139742,2.630000e+00
5,1.114080e-02,0.215352,0.769023,16.355094,2.780000e+00
6,1.845434e-02,0.359491,0.783787,16.714584,3.210000e+00
7,2.949311e-03,0.058721,0.786146,16.773305,3.230000e+00
8,4.032503e-03,0.080368,0.789372,16.853673,3.310000e+00
9,3.586081e-03,0.071757,0.792241,16.925430,3.400000e+00


In [44]:
pg_sin_cober = K_1["K"].iloc[-1]-K_1["St"].iloc[-1]
pg_Total_sin_cober = perdida_sin_cober+ K_1["Call"].iloc[-1]

pg_con_cober =  K_1["K"].iloc[-1]

-1.8760880965026416

Segunda tabla PUT (verificar)

In [28]:
def segunda_tabla_put(K_1):
    
    dif_delta1CALL = [K_1["Delta_put"][i+1]-K_1["Delta_put"][i] for i in range(1,len(K_1))]
    
    factor = [1.2 if K_1["Gamma"][i]>=0.8 else 0.8 if K_1["Gamma"][i]<=0.4 else 1 for i in range(1, len(K_1))]
    
    
    compra_activo=[K_1["St"].iloc[0]*K_1["Delta_put"].iloc[0]]
    
    for i in range(len(dif_delta1CALL)):
        compra_activo.append(K_1["St"][i+1]*dif_delta1CALL[i])
        
    
    Activo_acum = [K_1["Delta_put"].iloc[0]*factor[0]]

    for i in range(len(dif_delta1CALL)):
        Activo_acum.append(Activo_acum[i]+dif_delta1CALL[i]*factor[i])    
    
    
    costo_acum = [compra_activo[0]]

    for i in range(len(dif_delta1CALL)):
        costo_acum.append(costo_acum[i]+compra_activo[i+1])
        
    ganancia_diaria= [np.max((-K_1["St"].iloc[i]+K_1["K"].iloc[i]),0) for i in range(len(K_1))]
    
    costo_total=costo_acum[-1]
    
    primas=K_1["Put"]
    primas= -primas.iloc[-1]
    
    total=costo_total+primas
    
    no_cubrir=K_1["St"].iloc[-1]
    
    beneficio=no_cubrir-total
    
    f=np.zeros(len(dif_delta1CALL)+1)
    
    for i in range(len(dif_delta1CALL)):
        f[i+1]=dif_delta1CALL[i]
    
    lista = pd.DataFrame([f,compra_activo,Activo_acum,costo_acum,ganancia_diaria])
    
    lista = lista.T
    lista.rename(columns = {0:'Diff', 1:'compra_activo',2:'Activo_acum',3:'Acum_dinero',4:'Ganancia_diaria'}, inplace = True)
    
    
    return lista, costo_total,primas,total,no_cubrir,beneficio

def segunda_tabla_put(K_1):
    
    dif_delta1CALL = [K_1["Delta_put"][i+1]-K_1["Delta_put"][i] for i in range(1,len(K_1))]
    
    factor = [1.2 if K_1["Gamma"][i]>=0.8 else 0.8 if K_1["Gamma"][i]<=0.4 else 1 for i in range(1, len(K_1))]
    
    Activo_acum = [K_1["Delta_put"].iloc[0]*factor[0]]

    for i in range(len(dif_delta1CALL)):
        Activo_acum.append(Activo_acum[i]+dif_delta1CALL[i]*factor[i])
    
    costo=K_1["St"]*Activo_acum
    
    costo_acum = [costo.iloc[0]]

    for i in range(len(dif_delta1CALL)):
        costo_acum.append(costo_acum[i]+costo[i+1])
    
    costo_total=costo_acum[-1]
    
    primas=K_1["Put"]
    primas=primas.iloc[-1]#consultar signo
    
    total=costo_total+primas
    
    no_cubrir=K_1["St"].iloc[-1]
    
    beneficio=no_cubrir-total
    
    f=np.zeros(len(dif_delta1CALL)+1)
    
    for i in range(len(dif_delta1CALL)):
        f[i+1]=dif_delta1CALL[i]
    
    lista = pd.DataFrame([f,Activo_acum,costo,costo_acum])
    
    lista = lista.T
    lista.rename(columns = {0:'Diff', 1:'Activo_acum',2:'Costo',3:'Costo_acum'}, inplace = True)
    
    
    return lista, costo_total,primas,total,no_cubrir,beneficio

In [29]:
st_k1_put=segunda_tabla_put(K_1)
st_k2_put=segunda_tabla_put(K_2)
st_k3_put=segunda_tabla_put(K_3)
st_k4_put=segunda_tabla_put(K_4)
st_k5_put=segunda_tabla_put(K_5)

In [30]:
st_k1_put[0]

,Diff,compra_activo,Activo_acum,Acum_dinero,Ganancia_diaria
0,0.000000e+00,-7.487583,-3.586866e-01,-7.487583,-7.629395e-07
1,1.956544e-01,3.267429,-2.021631e-01,-4.220154,-9.499996e-01
2,1.206692e-01,2.129811,-1.056277e-01,-2.090344,-1.750001e+00
3,3.553340e-02,0.655591,-7.720103e-02,-1.434753,-2.050000e+00
4,4.663967e-02,0.874494,-3.988929e-02,-0.560259,-2.630000e+00
5,1.114080e-02,0.215352,-3.097665e-02,-0.344907,-2.780000e+00
6,1.845434e-02,0.359491,-1.621318e-02,0.014583,-3.210000e+00
7,2.949311e-03,0.058721,-1.385373e-02,0.073304,-3.230000e+00
8,4.032503e-03,0.080368,-1.062773e-02,0.153672,-3.310000e+00
9,3.586081e-03,0.071757,-7.758864e-03,0.225430,-3.400000e+00


In [31]:
st_k1_put=segunda_tabla_put(K_1)
st_k2_put=segunda_tabla_put(K_2)
st_k3_put=segunda_tabla_put(K_3)
st_k4_put=segunda_tabla_put(K_4)
st_k5_put=segunda_tabla_put(K_5)

In [32]:
K1_Final = {'Call':pd.concat([K_1.reset_index(), st_k1_call[0]], axis=1),
            'Put': pd.concat([K_1.reset_index(), st_k1_put[0]], axis=1)}

K2_Final = {'Call':pd.concat([K_2.reset_index(), st_k2_call[0]], axis=1),
            'Put': pd.concat([K_2.reset_index(), st_k2_put[0]], axis=1)}

K3_Final = {'Call':pd.concat([K_3.reset_index(), st_k3_call[0]], axis=1),
            'Put': pd.concat([K_3.reset_index(), st_k3_put[0]], axis=1)}

K4_Final = {'Call':pd.concat([K_4.reset_index(), st_k4_call[0]], axis=1),
            'Put': pd.concat([K_4.reset_index(), st_k4_put[0]], axis=1)}

K5_Final = {'Call':pd.concat([K_5.reset_index(), st_k5_call[0]], axis=1),
            'Put': pd.concat([K_5.reset_index(), st_k5_put[0]], axis=1)}

In [33]:
Ganancias1 = {'Call':st_k1_call[1:],
            'Put': st_k1_put[1:]}

Ganancias2 = {'Call': st_k2_call[1:],
            'Put': st_k2_put[1:]}

Ganancias3 = {'Call': st_k3_call[1:],
            'Put': st_k3_put[1:]}

Ganancias4 = {'Call': st_k4_call[1:],
            'Put': st_k4_put[1:]}

Ganancias5 = {'Call':st_k5_call[1:],
            'Put': st_k5_put[1:]}

In [34]:
print(k[0],Ganancias1[Eleccion])
print(k[1],Ganancias2[Eleccion])
print(k[2],Ganancias3[Eleccion])
print(k[3],Ganancias4[Eleccion])
print(k[4],Ganancias5[Eleccion])

16.7 (17.11985430127489, -1.3639124375549763, 15.755941863719915, 19.940000534057617, 4.184058670337702)
16.955 (17.221113807588974, -1.1299664661515187, 16.091147341437456, 19.940000534057617, 3.8488531926201617)
17.21 (17.334927040958558, -0.896029393204218, 16.438897647754338, 19.940000534057617, 3.501102886303279)
16.445 (17.03179351502642, -1.5978585160166237, 15.433934999009795, 19.940000534057617, 4.5060655350478225)
16.19 (16.956939738041406, -1.8318045949584878, 15.125135143082918, 19.940000534057617, 4.814865390974699)
